In [1]:
import sys
sys.path.append('../')

In [2]:
import seaborn as sns
from data_model import Individual
from sys_utils import load_model
import plotly.express as px

In [3]:
checkpoint_path = '../checkpoints_dev'

In [4]:
individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )



In [5]:
individuals_filtered = [x for x in individuals if x.regions != None]
individuals_filtered = [x for x in individuals_filtered if x.identifiers != None]

df_individuals = [
    {
        "wikidata_id": x.id.wikidata_id,
        "identifier":[y.name for y in x.identifiers],
        "region_code": x.regions,
         "year": x.impact_years,
    }
    for x in individuals_filtered
]


In [7]:
import pandas as pd

In [8]:
from data_model_region import Region
df = pd.DataFrame(df_individuals)
df = df.explode('region_code')

regions = load_model(
    Region, name=checkpoint_path + "/regions.jsonl"
)

df_regions = [
        {
            "region_code": x.code,
            "region_name": x.name,
        }
        for x in regions
    ]

df_regions = pd.DataFrame(df_regions)
df = pd.merge(df, df_regions, on = 'region_code')

In [1]:
import sys

sys.path.append('../')

from functions_env import DB_PATH
import sqlite3
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

In [2]:
conn = sqlite3.connect(DB_PATH)

In [3]:

df_individual = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_individual = df_individual[['individual_wikidata_id', 'birthyear']]
df_individual_region = pd.read_sql_query("SELECT * FROM individuals_regions", conn)
df_individual_region = df_individual_region[['individual_wikidata_id', 'region_name']].drop_duplicates()

In [6]:
df_identifiers = pd.read_sql_query("SELECT * FROM identifiers", conn)
df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''
df_identifiers['identifier_name'] = df_identifiers.apply(lambda x : x['identifier_name'] + ' ('  + x['country_name'] + ')' if x['country_name']!='' else x['identifier_name'], axis=1)
df_identifiers = df_identifiers.drop(['country_wikidata_id', 'count_records', 'identifier_url'], axis=1).drop_duplicates()

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_34841/1610848963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_identifiers['country_name'][df_identifiers['country_name'].isna()]=''


In [8]:
df_ind_identifiers = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
df_ind_identifiers = df_ind_identifiers.drop('identifier_name', axis=1)
df_ind_identifiers = pd.merge(df_ind_identifiers, df_identifiers, on = 'identifiers_wikidata_id', how = 'left')

df_final = pd.merge(df_individual, df_ind_identifiers, on = 'individual_wikidata_id', how = 'left')
df_final = pd.merge(df_final, df_individual_region, on = 'individual_wikidata_id')

In [11]:
region_name = 'Low countries'
min_year = 1300
max_year = 1850

In [42]:
df_country = df_final[df_final['region_name']=='Low countries']
df_country = df_country[(df_country['birthyear']>=min_year)&(df_country['birthyear']<=max_year)].reset_index(drop=True)
df_country["year"] = df_country["birthyear"].apply(lambda x : round(x/10)*10)
df_country = df_country.groupby(['year', 'identifiers_wikidata_id'])['individual_wikidata_id'].count().reset_index()

In [39]:
df_country = df_country[df_country['identifiers_wikidata_id']=='P1006']

,identifiers_wikidata_id,individual_wikidata_id
year,,
1300,P1006,2


In [9]:


df_country = df[df['region_name']==region_name]
df_country["year"] = df_country["year"].apply(
        lambda x: [year for year in range(int(x[0]), int(x[1]) + 10, 10)]
    )

df_country = df_country.explode('year').reset_index(drop=True)

top_identifiers = df_country[['wikidata_id','identifier']].explode('identifier').drop_duplicates().reset_index(drop=True)
top_identifiers = top_identifiers['identifier'].value_counts().reset_index().head(40)
top_identifiers = list(top_identifiers['index'])

df_country_fil = df_country.explode('identifier')
df_country_fil = df_country_fil[df_country_fil['identifier'].isin(top_identifiers)].reset_index(drop=True)


df_fig = df_country_fil.groupby(['identifier', 'year'])['wikidata_id'].count().reset_index()
df_fig = df_fig[(df_fig['year']<= max_year)&(df_fig['year']>= min_year)]

df_fig = df_fig.rename(columns = {'wikidata_id':'Number of Individuals'})



/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_25876/2564892623.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country["year"] = df_country["year"].apply(


In [15]:
fig = px.line(df_fig, x = 'year', 
              y = 'Number of Individuals', 
              color = 'identifier', 
              width = 2000, 
              height = 1000, 
              title = "Low Countres - Relations between identifiers", 
              template = 'plotly_white')

fig.update_xaxes(showgrid=False, showticklabels=True, zeroline=False)
fig.update_yaxes(showgrid=False, showticklabels=True, zeroline=False)
fig.update_layout(title_font=dict(size=30))

fig.update_layout(
    xaxis_title="",
    yaxis_title='Number of Individuals added')
import plotly
plotly.offline.plot(fig)
#fig.write_image(f"save_figures/identifiers_line.png", scale=7)

'temp-plot.html'

In [ ]:
import plotly

In [ ]:
plotly.offline.plot(fig)